In [ ]:
# import modules

import spacy
import textacy
import scattertext as st
import pandas as pd
import plotnine as p9 
from plotnine import ggplot, aes, geom_line, geom_point, stat_smooth, theme_classic

In [ ]:
# create corpus object --> define function

def get_texts_from_csv(f_csv, text_column):
    """
   Read dataset from a csv file and sequentially stream the rows,
    including metadata.
    """

    # read dataframe
    df = pd.read_csv(f_csv)

    # keep only documents that have text
    filtered_df = df[df[text_column].notnull()]
    
    # iterate over rows in dataframe
    for idx, row in filtered_df.iterrows():
        
        #read text and join lines (hard line-breaks)
        text = row[text_column].replace('\n', ' ')

        #use all columns as metadata, except the column with the actual text
        metadata = row.to_dict()
        del metadata[text_column]

        # return documents one after another (sequentially)
        yield (text, metadata)

In [ ]:
# create corpus object --> process documents and create corpus

# stream texts from a given folder
f_csv = '../KED2022/materials/data/dataset_speeches_federal_council_2019.csv'
texts = get_texts_from_csv(f_csv, text_column='text')

# load german language model
de = textacy.load_spacy_lang("de_core_news_sm")

# create corpus from processed documents
corpus_speeches_XY = textacy.Corpus(de, data=texts)

In [ ]:
# make Python understand gender
m = "male"
f = "female"
Geschlecht = [m,f]

In [ ]:
# function to filter by metadata gender (all speeches held by a female person)
def filter_func_female(doc):
    return doc._.meta.get("Geschlecht") == 'f'

# create new corpus after applying filter function
subcor_female = textacy.corpus.Corpus(de, data=corpus_speeches_XY.get(filter_func_female))

In [ ]:
# function to filter by metadata gender (all speeches held by a male person)
def filter_func_male(doc):
    return doc._.meta.get("Geschlecht") == 'm'

# create new corpus after applying filter function
subcor_male = textacy.corpus.Corpus(de, data=corpus_speeches_XY.get(filter_func_male))

In [ ]:
# export corpus as csv dataset --> See Code provided by other Teammember